<a href="https://colab.research.google.com/github/shreyajain4/WordLevelLangIdentificationJaech/blob/master/BTP_svm_graphemes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import argparse
import collections
import json
import logging
import numpy as np
import os
import shutil
import tensorflow as tf
import pandas as pd
import itertools
import re
import pickle
import random
import matplotlib
import csv
from sklearn.datasets.samples_generator import make_blobs 
matplotlib.rc('font', family='DejaVu Sans')
from matplotlib import pyplot
from nltk import ngrams
import numpy as np

In [0]:
sentences = []
words = []
labels = []
l = []
i = 0

w = []

length = []
# with open("drive/My Drive/dataset/POS Hindi English Code Mixed Tweets.tsv") as tsvfile:
#     tsvreader = csv.reader(tsvfile, delimiter="\t")
#     for line in tsvreader:
#       i = i+1
#       if(len(line)==3 and line[1]!='rest'):
#         length.append(len(Graphemes(line[0])))
# length=np.array(length)
# a=np.percentile(length,99.97)
with open("drive/My Drive/dataset/POS Hindi English Code Mixed Tweets.tsv") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t")
    for line in tsvreader:
      i = i+1
      if (len(line) == 3):
        if (len(line)>0 and (line[0]=='0' or line[0]=='')):
          words.append(w) 
          labels.append(l)
          w = []
          l = []
        elif(line[1]!='rest'):
          w.append(line[0])
          l.append(line[1])
      elif (len(line)==0 or (len(line)>0 and (line[0]=='0' or line[0]==''))):
          words.append(w) 
          labels.append(l)
          w = []
          l = []


In [0]:
legal_onsets = set()
for line in words:
  for w in line:
    w = w.lower()
    o = ""
    for char in w:
      if (char in 'aeiouy'):
        break
      o = o + char
    legal_onsets.add(o)

def is_legal_onset(onset):
  if (onset in legal_onsets):
    return True
  return False     

In [0]:
def Graphemes(w):
  segmented_word = []
  w = w.lower()
  o = ""
  i = 0
  vowel_left = 0
  for char in w:
    if (char in 'aeiouy'):
      vowel_left = vowel_left + 1 
  for char in w:
    if (char in 'aeiouy'):
      break
    o = o + char
    i = i + 1
  # onset.append(o)
  segmented_word.append(o)
  # if (i != 0):
    # segmented_word.append(w[:i])
    # last_indx = i

  while (i  < len(w)):
    n = ""
    while (i  < len(w)) and (w[i] in 'aeiouy'):
      vowel_left = vowel_left - 1
      n = n + w[i]
      i = i + 1
    # nucleus.append(n)
    segmented_word.append(n)
    o = ""
    c = ""
    if vowel_left == 0 :
      while (i < len(w)):
        c = c + w[i]
        i = i + 1 
    else:
      while (i  < len(w)) and (w[i] not in 'aeiouy'):
        o = o + w [i]
        i = i + 1

      # until onset is legal
      while (is_legal_onset(o) == False) :
        c = c + o[0]
        o = o[1:]
    if (o != ""):
      # onset.append(o)
      segmented_word.append(o)
      # segmented_word.append(w[last_indx:i])
      # last_indx = i
    if (c != ""):
      # coda.append(c)
      segmented_word.append(c)
  return segmented_word


In [0]:
max_word_len = 13

sentences = []
words = []
labels = []
l = []
i = 0
w = []

length = []
# with open("drive/My Drive/dataset/POS Hindi English Code Mixed Tweets.tsv") as tsvfile:
#     tsvreader = csv.reader(tsvfile, delimiter="\t")
#     for line in tsvreader:
#       i = i+1
#       if(len(line)==3 and line[1]!='rest'):
#         length.append(len(Graphemes(line[0])))
# length=np.array(length)
# a=np.percentile(length,99.97)
all_grams = set()
word_unique = dict()
with open("drive/My Drive/dataset/POS Hindi English Code Mixed Tweets.tsv") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t")
    for line in tsvreader:
      i = i+1
      if (len(line) == 3):
        if (len(line)>0 and (line[0]=='0' or line[0]=='')):
          words.append(w) 
          labels.append(l)
          
          w = []
          l = []
        elif(line[1]!='rest'):
          w.append(line[0])
          l.append(line[1])
          word_unique[line[0].lower()]=line[1]
          word=line[0]
          sixgrams = Graphemes(line[0].lower())
          for grams in sixgrams:
            
            all_grams.add(grams)

      elif (len(line)==0 or (len(line)>0 and (line[0]=='0' or line[0]==''))):
          words.append(w) 
          labels.append(l)
          w = []
          l = []


In [0]:
all_grams

{'',
 '\t',
 '\tr',
 '&',
 "'",
 "'d",
 "'ll",
 "'m",
 "'s",
 ')',
 '*',
 '-',
 '-j',
 '-r',
 '.',
 '/xn2',
 '1st',
 '4',
 '45',
 '4g',
 'a',
 'aa',
 'aaa',
 'aao',
 'aaoo',
 'aau',
 'aaya',
 'aaye',
 'aayi',
 'aayo',
 'ae',
 'ai',
 'aio',
 'aiy',
 'aiye',
 'aiyya',
 'ao',
 'au',
 'ay',
 'aya',
 'aye',
 'ayee',
 'ayei',
 'ayi',
 'ayya',
 'b',
 'b4',
 'bb',
 'bc',
 'bd',
 'bdd',
 'bh',
 'bhj',
 'bhn',
 'bhrn',
 'bht',
 'bj',
 'bk',
 'bl',
 'bn',
 'br',
 'brt',
 'brtl',
 'bs',
 'bt',
 'btc',
 'btw',
 'c',
 'cc',
 'ch',
 'ch*t',
 'chh',
 'chl',
 'cht',
 'ck',
 'cks',
 'cl',
 'cnd',
 'cntr',
 'cr',
 'cs',
 'ct',
 'cts',
 'cvd',
 'd',
 'dd',
 'ddng',
 'df',
 'dh',
 'dj\n',
 'dm',
 'dn',
 "dn't",
 'dnt',
 'dpnds',
 'dr',
 'ds',
 'e',
 'ea',
 'eau',
 'ee',
 'ei',
 'eiey',
 'eo',
 'eu',
 'ey',
 'eya',
 'eye',
 'eyo',
 'f',
 'f-',
 'ff',
 'fl',
 'fng',
 'fr',
 'frm',
 'fs',
 'ft',
 'g',
 'g**d',
 'gg',
 'gggg',
 'gh',
 'ghr',
 'ght',
 'ghts',
 'gn',
 'gnst',
 'gr',
 'grt',
 'grv',
 'gs',
 'gt',

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
ngram_voc=dict()
def vocab():
  i=0
  for word in all_grams:
    ngram_voc[word.lower()]=i
    i=i+1
vocab()
ngram_voc

out_vocab=dict()
out_vocab['hi']=0
out_vocab['en']=1
len(ngram_voc)

403

In [0]:
len(ngram_voc)

403

In [0]:
x=np.zeros((len(word_unique),len(ngram_voc)))
y=np.zeros((len(word_unique)))

In [0]:
i=0
ww = []
p=[]
for w, l in word_unique.items():
  word=w
  sixgrams = Graphemes(w)
  # if (i<5):
  #   print (list(word.lower()))
  for grams in sixgrams:
    
    x[i][ngram_voc[grams]]=1
    if(i==3):
      p.append(ngram_voc[grams])
  if l=='en':
    y[i]=1
  i=i+1
p

[96, 187, 386, 192]

In [0]:
# word_unique


In [0]:
x[3][97]

0.0

In [0]:
# p=x[0]==np.zeros((1,len(ngram_voc)))
# p
k=0
indeces = []
for z in x[0]:
  if(z==1):
    indeces.append(k)
  k = k+1
indeces

[]

In [0]:
# X, Y = make_blobs(n_samples=len(word_unique), centers=2, random_state=0, cluster_std=0.40) 

In [0]:
from sklearn.svm import SVC # "Support Vector Classifier" 
clf = SVC(kernel='linear') 
len(word_unique)

3113

In [0]:
clf.fit(x[0:2700], y[0:2700]) 


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
#new_word = np.zeros(1,len(ngram_voc)))
clf.score(x[2700:],y[2700:])


0.8208232445520581